### Purpose of File
This Notebook is mainly for three purposes
1. Calculate Column Pairwise Similarity 
1. Agregate aforementioned pairwise similarity by solution
1. Calculate similarity to Problem Statement in MIT Solve

In [2]:
import pandas as pd
from numpy import dot, mean
from numpy.linalg import norm
from itertools import combinations
from encode import encoder

In [3]:
## Calculation of Cosine similarity
def cosine_similarity(a,b):
    try:
        cos_sim = dot(a, b)/(norm(a)*norm(b))
        return cos_sim
    except:
        return None

In [4]:
## Get Combinations of Relevant Columns
df = pd.read_parquet("./Data/encoded.parquet")
columns = df.columns[3:]
combination_pairs = list(combinations(columns,2))

In [5]:
## Iterate through the Pairs of Columns
for c in combination_pairs:
    df[c[0] + "_" + c[1]] = df.apply(lambda x: cosine_similarity(x[c[0]], x[c[1]]), axis=1 )


df = df.drop(columns, axis=1)
df.to_csv("./Data/ColumnPairwiseSimilarity.csv", index=None)

/var/folders/3k/zk90yh5d7l72d_pjvqksx7hr0000gq/T/ipykernel_27751/2979081075.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[c[0] + "_" + c[1]] = df.apply(lambda x: cosine_similarity(x[c[0]], x[c[1]]), axis=1 )
/var/folders/3k/zk90yh5d7l72d_pjvqksx7hr0000gq/T/ipykernel_27751/2979081075.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[c[0] + "_" + c[1]] = df.apply(lambda x: cosine_similarity(x[c[0]], x[c[1]]), axis=1 )
/var/folders/3k/zk90yh5d7l72d_pjvqksx7hr0000gq/T/ipykernel_27751/2979081075.py:3: PerformanceWarning

In [6]:
## Reshape the dataframe
## Change the pairwise columns from columns to rows
## Calculate the Mean
sim = df.drop(["Challenge", "Status"], axis=1) \
        .melt(id_vars="ID").groupby("variable") \
        .agg(mean) \
        .reset_index()

## Sort from MOST similar to LEAST
sim = sim.sort_values("value", ascending=False) \
         .drop(["ID"], axis=1)

sim.columns = ["CombinationPair", "CosineSimilarity"]

sim.to_csv("./Data/AggregatedColumnPairwiseSimilarity.csv", index=None)

In [7]:
## Get the Problem Statement
df = pd.read_parquet("./Data/encoded.parquet")
ps = pd.read_csv("./Data/ProblemStatements.csv")

## Merge the Problem Statement to appropriate Challenge
ps = ps[["Challenge", "Problem_Statement"]].dropna()
ps["Problem_Statement"] = ps["Problem_Statement"].apply(encoder)
df = df.merge(ps, how="left", on = "Challenge")

In [8]:
## Iterate through all columns to calculate similarity
for c in columns:
    df[c] = df.apply(lambda x: cosine_similarity(x[c], x["Problem_Statement"]), axis=1)

## Write to CSV
df.drop(["Problem_Statement"], axis = 1).to_csv("./Data/SimilarityToPS.csv", index = None)